In [1]:
# 导入椭圆曲线算法
from ecdsa import SigningKey, SECP256k1, VerifyingKey, BadSignatureError
import binascii
import base64
from hashlib import sha256


class Wallet:
    """
        钱包
    """
    def __init__(self):
        """
            钱包初始化时基于椭圆曲线生成一个唯一的秘钥对，代表区块链上一个唯一的账户
        """
        self._private_key = SigningKey.generate(curve=SECP256k1)
        self._public_key = self._private_key.get_verifying_key()

    @property
    def address(self):
        """
            这里通过公钥生成地址
        """
        h = sha256(self._public_key.to_pem())
        return base64.b64encode(h.digest())

    @property
    def pubkey(self):
        """
            返回公钥字符串
        """
        return self._public_key.to_pem()

    def sign(self, message):
        """
            生成数字签名
        """
        h = sha256(message.encode('utf8'))
        return binascii.hexlify(self._private_key.sign(h.digest()))

    
def verify_sign(pubkey, message, signature):
    """
        验证签名
    """
    verifier = VerifyingKey.from_pem(pubkey)
    h = sha256(message.encode('utf8'))
    return verifier.verify(binascii.unhexlify(signature), h.digest())

In [2]:
import json

class Transaction:
    """
    交易的结构
    """
    def __init__(self, sender, recipient, amount):
        """
            初始化交易，设置交易的发送方、接收方和交易数量
        """
        if isinstance(sender, bytes):
            sender = sender.decode('utf-8')
        self.sender = sender            # 发送方
        if isinstance(recipient, bytes):
            recipient = recipient.decode('utf-8')
        self.recipient = recipient      # 接收方
        self.amount = amount            # 交易数量
        
    def set_sign(self, signature, pubkey):
        """
            为了便于验证这个交易的可靠性，需要发送方输入他的公钥和签名
        """
        self.signature = signature      # 签名
        self.pubkey = pubkey            # 发送方公钥
        
    def __repr__(self):
        """
            交易大致可分为两种，一是挖矿所得，而是转账交易
            挖矿所得无发送方，以此进行区分显示不同内容
        """
        if self.sender:
            s = "从 %s 转至 %s %d个加密货币" % (self.sender, self.recipient, self.amount)
        else:
            s = "%s 挖矿获取%d个加密货币" % (self.recipient, self.amount)
        return s


class TransactionEncoder(json.JSONEncoder):
    """
    定义Json的编码类，用来序列化Transaction
    """
    def default(self, obj):
        if isinstance(obj, Transaction):
            return obj.__dict__
        else:
            return json.JSONEncoder.default(self, obj)

In [3]:
import hashlib
from datetime import datetime


class Block:
    """
        区块结构
            prev_hash:      父区块哈希值
            transactions:           交易对
            timestamp:      区块创建时间
            hash:           区块哈希值
            Nonce:        随机数
    """
    def __init__(self, transactions, prev_hash):
        # 将传入的父哈希值和数据保存到类变量中
        self.prev_hash = prev_hash    
        self.transactions = transactions
        # 获取当前时间
        self.timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        # 设置Nonce和哈希的初始值为None
        self.nonce = None
        self.hash = None
        
    def __repr__(self):
        return "区块内容：%s\n哈希值: %s" % (json.dumps(self.transactions), self.hash)

In [4]:
DIFFICULTY = 5

class ProofOfWork:
    """
        工作量证明
    """
    def __init__(self, block, miner, difficult=5):
        self.block = block
        
        # 定义工作量难度，默认为5，表示有效的哈希值以5个“0”开头
        self.difficulty = DIFFICULTY

        self.miner = miner
        # 添加挖矿奖励
        self.reward_amount = 1

    def mine(self):
        """
            挖矿函数
        """
        i = 0
        prefix = '0' * self.difficulty
        
        
        # 添加奖励
        t = Transaction(
                sender="",
                recipient=self.miner.address,
                amount=self.reward_amount,
            )
        sig = self.miner.sign(json.dumps(t, cls=TransactionEncoder))
        t.set_sign(sig, self.miner.pubkey)
        self.block.transactions.append(t)

        while True:
            message = hashlib.sha256()
            message.update(str(self.block.prev_hash).encode('utf-8'))
            # 更新区块中的交易数据
            # message.update(str(self.block.data).encode('utf-8'))
            message.update(str(self.block.transactions).encode('utf-8'))
            message.update(str(self.block.timestamp).encode('utf-8'))
            message.update(str(i).encode("utf-8"))
            digest = message.hexdigest()
            if digest.startswith(prefix):
                self.block.nonce = i
                self.block.hash = digest
                return self.block
            i += 1

In [5]:
class BlockChain:
    """
        区块链结构体
            blocks:        包含的区块列表
    """
    def __init__(self):
        self.blocks = []

    def add_block(self, block):
        """
        添加区块
        """
        self.blocks.append(block)

In [6]:
import socket  
import threading

# 定义一个全局列表保存所有节点
NODE_LIST = []

class Node(threading.Thread):
    def __init__(self, name, port, host="localhost"):
        threading.Thread.__init__(self, name=name)
        self.host = host       #  服务器地址，本地电脑都设为localhost
        self.port = port        # 每个节点对应一个唯一的端口号
        self.name = name    # 唯一的节点名称
        self.wallet = Wallet()
        self.blockchain = None    # 用来存储一个区块链副本
        
    def run(self):
        """
            节点运行
        """
        self.init_blockchain()    # 初始化区块链
        
        # 在指定端口进行监听
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)  
        sock.bind((self.host, self.port))  
        NODE_LIST.append({
            "name": self.name,
            "host": self.host,
            "port": self.port
        })
        sock.listen(10) 
        print(self.name, "运行中...")
        while True:       # 不断处理其他节点发送的请求
            connection,address = sock.accept()  
            try:
                print(self.name, "处理请求内容...")
                self.handle_request(connection)
            except socket.timeout:  
                print('超时!')
            except Exception as e:
                print(e, )
            connection.close()
    
    def handle_request(self, connection):
        data = []
        while True:    # 不断读取请求数据直至读取完成
            buf = connection.recv(PER_BYTE)  
            if not buf: # 若读取不到新的数据则退出
                break
            data.append(buf)
            if len(buf) < PER_BYTE:   # 若读取到的数据长度小于规定长度，说明数据读取完成，退出
                break
        t = pickle.loads(b''.join(data))
        if isinstance(t, Transaction):  # 如果是新区块类型类型消息
            print("处理交易请求...")
            if verify_sign(t.pubkey, 
                  str(t),
                   t.signature):
    
                # 验证交易签名没问题，生成一个新的区块
                print(self.name, "验证交易成功")
                new_block = Block(transactions=[t], prev_hash="")
                print(self.name, "生成新的区块...")
                w = ProofOfWork(new_block, self.wallet)
                block = w.mine()
                print(self.name, "将新区块添加到区块链中")
                self.blockchain.add_block(block)
                print(self.name, "将新区块广播到网络中...")
                self.broadcast_new_block(block)
            else:
                print(self.name, "交易验证失败！")
        elif isinstance(t, Block):
            print("处理新区块请求...")
            if self.verify_block(t):
                print(self.name, "区块验证成功")
                self.blockchain.add_block(t)
                print(self.name, "添加新区块成功")
            else:
                print(self.name, "区块验证失败!")
        else:  # 如果不是新区块消息，默认为初始化消息类型，返回本地区块链内容
            connection.send(pickle.dumps(self.blockchain))
            
    def verify_block(self, block):
        """
            验证区块有效性
        """
        message = hashlib.sha256()
        message.update(str(block.prev_hash).encode('utf-8'))
        # 更新区块中的交易数据
        # message.update(str(self.block.data).encode('utf-8'))
        message.update(str(block.transactions).encode('utf-8'))
        message.update(str(block.timestamp).encode('utf-8'))
        message.update(str(block.nonce).encode('utf-8'))
        digest = message.hexdigest()

        prefix = '0' * DIFFICULTY
        return digest.startswith(prefix)
            
    def broadcast_new_block(self, block):
        """
            将新生成的区块广播到网络中其他节点
        """
        for node in NODE_LIST:
            host =node['host']
            port = node['port']
            
            if host == self.host and port == self.port:
                print(self.name, "忽略自身节点")
            else:
                print(self.name, "广播新区块至 %s" % (node['name']))
                sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                sock.connect((host, port))    # 连接到网络中的节点
                sock.send(pickle.dumps(block))   # 发送新区块
                sock.close()          # 发送完成后关闭连接
                
    def init_blockchain(self):
        """
            初始化当前节点的区块链
        """
        if NODE_LIST:                # 若当前网络中已存在其他节点，则从第一个节点从获取区块链信息
            host = NODE_LIST[0]['host']
            port = NODE_LIST[0]['port']
            name = NODE_LIST[0]["name"]
            print(self.name, "发送初始化请求 %s" % (name))
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.connect((host, port))    # 连接到网络中的第一个节点
            sock.send(pickle.dumps('INIT'))   # 发送初始化请求
            data = []
            while True:          # 读取区块链信息，直至完全获取后退出
                buf = sock.recv(PER_BYTE)
                if not buf: 
                    break
                data.append(buf)
                if len(buf) < PER_BYTE:
                    break
            sock.close()   # 获取完成后关闭连接
            
            # 将获取的区块链信息赋值到当前节点
            self.blockchain = pickle.loads(b''.join(data))
            print(self.name, "初始化完成.")
        else:
            # 如果是网络中的第一个节点，初始化一个创世区块
            block = Block(transactions=[], prev_hash="")
            w = ProofOfWork(block, self.wallet)
            genesis_block = w.mine()
            self.blockchain = BlockChain()
            self.blockchain.add_block(genesis_block)
            print("生成创世区块")
    
    def submit_transaction(self, transaction):
         for node in NODE_LIST:
            host =node['host']
            port = node['port']
            
            if host == self.host and port == self.port:
                print(self.name, "忽略自身节点")
            else:
                print(self.name, "广播新区块至 %s:%s" % (self.host, self.port))
                sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)  
                sock.connect((node["host"], node["port"]))  
                sock.send(pickle.dumps(transaction)) 
                sock.close()
        
    def get_balance(self):
        balance = 0
        for block in self.blockchain.blocks:
            for t in block.transactions:
                if t.sender == self.wallet.address.decode():
                    balance -= t.amount
                elif t.recipient == self.wallet.address.decode():
                    balance += t.amount
        print("当前拥有%.1f个加密货币" % (balance))
    
    def print_blockchain(self):
        print("区块链包含区块个数: %d\n" % len(self.blockchain.blocks))
        for block in self.blockchain.blocks:
            print("上个区块哈希：%s" % block.prev_hash)
            print("区块内容：%s" % block.transactions)
            print("区块哈希：%s" % block.hash)
            print("\n") 

In [7]:
# 初始化节点1

node1 = Node("节点1", 8000)

In [8]:
node1.start()

生成创世区块
节点1 运行中...


In [9]:
node1.print_blockchain()

区块链包含区块个数: 1

上个区块哈希：
区块内容：[+welFYuuTR5XxItoPwrm7Pue+EhBVJe4HMQ5lpbWstw= 挖矿获取1个加密货币]
区块哈希：00000253dbf5699116a1d8d948fb8da76bb12f41f696a9ebe87c2624ecd55c2b




In [10]:
node2 = Node("节点2", 8001)

In [11]:
node2.start()

节点2 发送初始化请求 节点1
节点1 处理请求内容...
节点2 初始化完成.
节点2 运行中...


In [12]:
node2.print_blockchain()

区块链包含区块个数: 1

上个区块哈希：
区块内容：[+welFYuuTR5XxItoPwrm7Pue+EhBVJe4HMQ5lpbWstw= 挖矿获取1个加密货币]
区块哈希：00000253dbf5699116a1d8d948fb8da76bb12f41f696a9ebe87c2624ecd55c2b




In [13]:
node1.get_balance()

当前拥有1.0个加密货币


In [14]:
node2.get_balance()

当前拥有0.0个加密货币


In [15]:
new_transaction = Transaction(
    sender=node1.wallet.address,
    recipient=node2.wallet.address,
    amount=0.3
)
sig = node1.wallet.sign(str(new_transaction))
new_transaction.set_sign(sig, node1.wallet.pubkey)

In [16]:
node1.submit_transaction(new_transaction)

节点1 忽略自身节点
节点1 广播新区块至 localhost:8000
节点2 处理请求内容...
处理交易请求...
节点2 验证交易成功
节点2 生成新的区块...
节点2 将新区块添加到区块链中
节点2 将新区块广播到网络中...
节点2 广播新区块至 节点1
节点1 处理请求内容...
节点2处理新区块请求...
节点1 区块验证成功
节点1  忽略自身节点
添加新区块成功


In [17]:
node1.print_blockchain()

区块链包含区块个数: 2

上个区块哈希：
区块内容：[+welFYuuTR5XxItoPwrm7Pue+EhBVJe4HMQ5lpbWstw= 挖矿获取1个加密货币]
区块哈希：00000253dbf5699116a1d8d948fb8da76bb12f41f696a9ebe87c2624ecd55c2b


上个区块哈希：
区块内容：[从 +welFYuuTR5XxItoPwrm7Pue+EhBVJe4HMQ5lpbWstw= 转至 6SM6EU514L4h0W4WufIKR2ISva7Lz/DN1C0OWBYhqQ8= 0个加密货币, 6SM6EU514L4h0W4WufIKR2ISva7Lz/DN1C0OWBYhqQ8= 挖矿获取1个加密货币]
区块哈希：00000c7e955e08c0ea8312982f381f5fa73640079a0bb2e1e54503d5ec618ee7




In [18]:
node2.print_blockchain()

区块链包含区块个数: 2

上个区块哈希：
区块内容：[+welFYuuTR5XxItoPwrm7Pue+EhBVJe4HMQ5lpbWstw= 挖矿获取1个加密货币]
区块哈希：00000253dbf5699116a1d8d948fb8da76bb12f41f696a9ebe87c2624ecd55c2b


上个区块哈希：
区块内容：[从 +welFYuuTR5XxItoPwrm7Pue+EhBVJe4HMQ5lpbWstw= 转至 6SM6EU514L4h0W4WufIKR2ISva7Lz/DN1C0OWBYhqQ8= 0个加密货币, 6SM6EU514L4h0W4WufIKR2ISva7Lz/DN1C0OWBYhqQ8= 挖矿获取1个加密货币]
区块哈希：00000c7e955e08c0ea8312982f381f5fa73640079a0bb2e1e54503d5ec618ee7




In [19]:
node1.get_balance()

当前拥有0.7个加密货币


In [20]:
node2.get_balance()

当前拥有1.3个加密货币
